In [2]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
import pandas as pd
import numpy as np
import scipy.io as sio

In [ ]:
  # Check and set up CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

NameError: name 'device' is not defined

In [ ]:
# Load the CSV file
file_path = "./csv_xls_files/HH_design_mat.csv"  # Replace with your file path
df = pd.read_csv(file_path)
df.head()

,Section,Text,Start_time_section,End_time_section,Section_offset,Start_time_combined,End_time_combined,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,0,henry,1.41,1.70,0.0,1.41,1.70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,huggins,1.70,2.27,0.0,1.70,2.27,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,was,2.27,2.46,0.0,2.27,2.46,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,in,2.46,2.62,0.0,2.46,2.62,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,the,2.62,2.68,0.0,2.62,2.68,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Ensure all values in the 'word' column are strings and handle NaN values
# df['Story'] = df['Text'].fillna('').astype(str)

# story_words = df['Text'].to_list()
# Extract the 'word' column and concatenate into a story
story_text = ' '.join(df['Text'])

# Print or save the story
print(f"total word: {len(story_text.split())}. Text: {story_text}")

total word: 2095. Text: henry huggins was in the third grade his hair looked like a scrubbing brush and most of his grown up front teeth were in he lived with his mother and father in a square white house on klickitat street except for having his tonsils out when he was six and breaking his arm falling out of a cherry tree when he was seven nothing much had happened to henry i wish something exciting would happen henry often thought but nothing very exciting ever happened to henry at least not until one wednesday afternoon in march every wednesday after school henry rode downtown on the bus to go swimming at the gym after he swam for an hour he got on the bus again and rode home just in time for dinner it was fun but not really exciting when henry left the gym on this particular wednesday he stopped to watch a man tear down a circus poster then with three nickels and one dime in his pocket he went to the corner drugstore to buy a chocolate ice cream cone he thought he would eat the ice

In [ ]:

def extract_word_activations(story, model_name='gpt2'):
    # Load pre-trained GPT-2 model and tokenizer on CUDA
    model = GPT2Model.from_pretrained(model_name).to(device)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model.eval()

    # Split story into words
    words = story.split()
    
    # Initialize output activation matrix
    word_activations = torch.zeros(len(words), 768).cuda()
    
    # Process each word
    for i, word in enumerate(words):
        # Tokenize with previous context (up to 1024 tokens)
        context = ' '.join(words[max(0, i-1024//2):i])
        # inputs = tokenizer(context, return_tensors='pt', truncation=True, max_length=1024).to(device)
        
        # Add current word to input
        current_input = tokenizer(context + ' ' + word, return_tensors='pt', truncation=True, max_length=1024).to(device)
        
        # Get hidden states
        with torch.no_grad():
            outputs = model(**current_input, output_hidden_states=True)
            
        # Extract the 8th layer's hidden state for the last token (1024th position)
        # activation = outputs.hidden_states[8][0, -1, :]
        activation = outputs.hidden_states[6][0, -1, :]
        word_activations[i] = activation
    
    return word_activations.cpu()  # Move back to CPU for further processing


In [ ]:
layer_activations = extract_word_activations(story_text)
layer_6_full_activations = layer_activations[-1957:,:]
# print(f"Full story activation matrix shape: {layer_7_full_activations.shape}")
# torch.save(layer_7_full_activations, 'full_story_layer_7_activations.pt')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
output_path = './design_matrix/activation_gpt2/activation_baseline_6.mat'
# Save to .mat file
sio.savemat(output_path, {'activations': layer_6_full_activations})

In [ ]:
def normalize_columns(activations):
    # Normalize each column to [0,1] range
    normalized_activations = (activations - activations.min(axis=1, keepdims=True)) / \
                              (activations.max(axis=1, keepdims=True) - activations.min(axis=1, keepdims=True))
    
    return normalized_activations


In [ ]:
mat_file_path = './design_matrix/activation_gpt2/activation_baseline_6.mat'
# Load .mat file
mat_contents = sio.loadmat(mat_file_path)

# Assuming the matrix is stored under 'activations' key
activations = mat_contents['activations']

norm_activations = normalize_columns(activations)

In [ ]:
output_path = './design_matrix/activation_gpt2/norm_activation_baseline_6.mat'
# Save to .mat file
sio.savemat(output_path, {'activations': norm_activations})